In [15]:
import os
import numpy as np
import pandas as pd
from custom import utils
from collections import Counter
import inspect
import scipy.io

### Function for dealing with the handedness covariate
This function performs categorical imputation. 
    This decision is made based on the fact that ~90 out of 1000+ 
    handed values are missing. That small percentage is imputed 
    with the most frequent handedness score "Right". There's also
    a very small percentage of non right non left values that 
    are a mix of "Both", "Mixed", "Either", "Ambidextrous", these
    are all imputed to just "Both"

In [5]:
def most_handed(data):
    """data: pd.DataFrame"""
    counts = Counter(data)
    most = max(counts.items())[0]
    data = data.copy().fillna(0)
    data[data == 0] = 'Right'
    both = ['Both','Mixed','Either','Ambidextrous']
    for hand in both:
        data[data == hand] = 'Both'
    return data

## Preprocessing function
Inputs are paths to data, sMRI features and SNPs respectively. "bcvar" is a list of covariates: ['SEX', 'AGE_MRI', 'EstimatedTotalIntraCranialVol', 'STUDY']. "brain_cols" is a numpy array of feature names to subset the sMRI data. 
First thing that happens is I get a boolen array of where the brain data contains only Controls or Schizophrenics in the Group feature. Both datasets are subsetted by this boolean array, row wise. It's important to note that the datasets loaded are already in the same order rowwise. Then I create 2 sets of covariate matrices and concatenate them into one. The first set contains AGE, SEX, and ICV(EstimatedTotalIntraCranialVol), the second one is a one hot encoded matrix of handedness, the third is a one hot encoded matrix of site(i.e, study). The first step of the analysis does not use the response variable but I safe it to apply stratifiedKFold cross-validation. Two dictionaries are returned, one contains keys and numpy arrays. The name of the keys are required inputs for the first part of the analysis. The dictionary containing the column headers is not required but is saved for later uses. 

In [ ]:
def preprocess(brain_path, snp_path, bcvar, brain_cols):
    """brain_path: String, snp_path: String, bcvar: list, 
    brain_cols: list or np.ndarray. 
    """
    # load data
    brain_data = pd.read_hdf(brain_path)
    snp_data = pd.read_hdf(snp_path)
    # get the group status
    gr = brain_data.GROUP.values
    cnt_scz = np.logical_or(gr == 'Control', gr == 'Schizophrenia')
    # subset by indexes cnt_scz
    brain_data = brain_data.iloc[cnt_scz, :]
    snp_data = snp_data.iloc[cnt_scz, :]
    # create set of covariates
    icv = 'EstimatedTotalIntraCranialVol'
    cov_set1 = pd.DataFrame(
        data=np.hstack((snp_data.SEX.values[:, None],
                        brain_data.AGE_MRI.values[:, None],
                        brain_data[icv].values[:, None])),
        columns=['SEX','AGE','EstimatedTotalIntraCranialVol'])
    cov_set1 = cov_set1.fillna(0)
    cov_set1[cov_set1.AGE == 0] = cov_set1.AGE.mean()
    cov_site = utils.make_non_singular(utils.encoder(brain_data.STUDY.values))
    cov_site_cols = ['site{}'.format(i) for i in range(cov_site.shape[1])]
    cov_site = pd.DataFrame(data=cov_site, columns=cov_site_cols)
    cov_hand = utils.encoder(most_handed(brain_data.HANDED))
    cov_hand_cols = ['handed{}'.format(i) for i in range(cov_hand.shape[1])]
    cov_hand = pd.DataFrame(data=cov_hand, columns=cov_hand_cols)
    cvars = pd.concat([cov_set1, cov_site, cov_hand], axis=1)
    y = np.array([0 if i == 'Control' else 1 for i in brain_data.GROUP.values])
    return {'Z': cvars.values, 
            'I': brain_data[brain_cols].values, 
            'G': snp_data.iloc[:, 1:-5].values,
            'colnames': snp_data.iloc[:, 1:-5].columns.values,
            'y':y}, {'Z_cols':cvars.columns.values,
                     'I_cols':brain_cols,
                     'G_cols': snp_data.iloc[:, 1:-5].columns.values}

# Helper functions
These two functions assist in the analysis. save_preprocessed saves the data to disk that are outputed from the preprocess function above. The paths to where the files are written on disk are returned. This is because I'll be using nipype so to make life easier data isn't passed when interfacing with nipype nodes - just the path to where the data lives. I then load the data using the paths. The cv_maker function creates k-fold stratified cross validation indices and saves them. These indices are used in the matlab script to load the correct subsets of data. 

In [22]:
def save_preprocessed(preproc_data_dict, preproc_data_dict_col, save_path, dn, cn):
    """preproc_data_dict: dictionary object returned from
    the preprocessing function, (the first - zeroth value of the return) 
    preproc_data_dict_col: dintionary object returned from
    the preprocessing function, (the second - first value of the return)
    save_path: string - base path for saving the dictionaries
    dn: string - name for saving the data dictionary 
    cn: string - name for saving the column header dictionary
    """
    save_dict = os.path.join(save_path, dn)
    save_cols = os.path.join(save_path, cn)
    scipy.io.savemat(save_dict, mdict=preproc_data_dict)
    utils.save_pickle(save_cols, preproc_data_dict_col)
    return save_dict, save_cols

def cv_maker(data_path, save_path):
    import scipy.io
    from sklearn.model_selection import StratifiedKFold
    X = scipy.io.loadmat(data_path)['I']
    y = scipy.io.loadmat(data_path)['y'][0]
    cv = StratifiedKFold(n_splits=10, random_state=3)
    train_idx, test_idx = {}, {}
    for idx, (train, test) in enumerate(cv.split(X, y)):
        train_idx['train_{}'.format(idx + 1)] = train + 1
        test_idx['test_{}'.format(idx + 1)] = test + 1
    scipy.io.savemat(save_path, mdict={"train":train_idx, "test":test_idx})
    return save_path

# Create the input data, save the CV indices
The step below runs the functions I've made above

In [24]:
headers_dir = "/storage/gablab001/data/genus/fs_cog/pred_diag/column_headers"
brain_cols = np.genfromtxt(os.path.join(headers_dir, "XB"), dtype=str)
brain_path = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/brain_N1547_P5927_matched.hdf5py"
snp_path = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/genomic_N1547_P100006_matched.hdf5py"
bcv = ['SEX', 'AGE_MRI', 'EstimatedTotalIntraCranialVol', 'STUDY']
all_data, all_cols = preprocess(brain_path, snp_path, bcv, brain_cols)
path_for_save = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets"
for_cv, _ = save_preprocessed(all_data, all_cols, path_for_save, "brain_gene.mat","brain_gene_cols.pkl")
cv_path = cv_maker(for_cv, os.path.join(path_for_save, "cv_idx.mat"))

In [25]:
from nipype import Function, Node, Workflow, IdentityInterface

# Bayesian analysis - "step 1"
Below I create the workflow that I use with nipype, create the nipype wrapper nodes to wrap functions that will go into the nipype graph, and then submit the jobs. Due to the nature of the analysis we are parallelizing over the feature space in the sMRI data. That is - one job per feature, on top of that we are parallelizing the cross validation step. In total this means there are (170*10) jobs that need to be submitted. For a single user in my experience that's too many jobs for the Openmind cluster so I limit the amount of jobs that can be submitted at a time. 

In [27]:
CV_maker = Node(interface=Function(
    input_names = ['data_path', 'save_path'],
    output_names = ['save_path'],
    function = cv_maker
), name = 'CV_maker')

#CV_maker.inputs.data_path = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/brain_gene.mat"
#CV_maker.inputs.save_path = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/cv_idx.mat"


wf = Workflow(name='brain_bcv')
wf.base_dir = "/om/scratch/Tue/ysa"

Iternode = Node(IdentityInterface(fields=['col_idx', 'cv_idx']), name = 'Iternode')
Iternode.iterables = [('col_idx', np.arange(170) + 1), ('cv_idx', np.arange(10) + 1)]

def run_bayes(in_file, cv_file, cv_idx, col_idx, out_file):
    import cPickle as pickle
    import numpy as np
    import os
    import nipype.interfaces.matlab as Matlab
    def outnames(col, out):
        return os.path.join(out, '{}.mat'.format(col))
    headers_dir = "/storage/gablab001/data/genus/fs_cog/pred_diag/column_headers"
    col_names = np.genfromtxt(os.path.join(headers_dir, "XB"), dtype=str)
    col_save_name = col_names[col_idx - 1] + "_{}_{}_BF".format(cv_idx, col_idx)
    with open("/storage/gablab001/data/genus/GIT/genus/bayes/matlab/bayes_reg.m", "r") as src:
        script = src.read().replace("\n", "")
    mat_file = outnames(in_file[:-4]+'_'+col_save_name, out_file)
    matlab = Matlab.MatlabCommand()
    matlab.inputs.script = script.format(in_file, cv_file, cv_idx, col_idx, mat_file)
    res = matlab.run()
    return mat_file

Run_bayes = Node(interface=Function(
    input_names = ['in_file', 'cv_file','cv_idx',
                   'col_idx','out_file'],
    output_names = ['mat_file'],
    function = run_bayes
), name='Run_bayes')

Run_bayes.inputs.in_file = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/brain_gene.mat"
Run_bayes.inputs.cv_file = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/cv_idxs.mat"
Run_bayes.inputs.out_file = "/storage/gablab001/data/genus/GIT/genus/bayes/results/bayes_factor"

wf.connect(Iternode, 'cv_idx', Run_bayes, 'cv_idx')
wf.connect(Iternode, 'col_idx', Run_bayes, 'col_idx')
wf.run(plugin='SLURM', plugin_args={'sbatch_args':'--mem=4G -t 23:00:00', 'max_jobs': 150})

170725-13:49:49,361 workflow INFO:
	 Workflow brain_bcv settings: ['check', 'execution', 'logging']
170725-13:49:54,429 workflow INFO:
	 Running in parallel.
170725-13:49:54,465 workflow INFO:
	 Pending[0] Submitting[150] jobs Slots[150]
170725-13:49:54,466 workflow INFO:
	 Submitting: Run_bayes.a1457 ID: 0
170725-13:49:54,554 workflow INFO:
	 Finished submitting: Run_bayes.a1457 ID: 0
170725-13:49:54,555 workflow INFO:
	 Submitting: Run_bayes.a0317 ID: 1
170725-13:49:54,641 workflow INFO:
	 Finished submitting: Run_bayes.a0317 ID: 1
170725-13:49:54,643 workflow INFO:
	 Submitting: Run_bayes.a1527 ID: 2
170725-13:49:54,728 workflow INFO:
	 Finished submitting: Run_bayes.a1527 ID: 2
170725-13:49:54,730 workflow INFO:
	 Submitting: Run_bayes.a0518 ID: 3
170725-13:49:54,817 workflow INFO:
	 Finished submitting: Run_bayes.a0518 ID: 3
170725-13:49:54,819 workflow INFO:
	 Submitting: Run_bayes.a0002 ID: 4
170725-13:49:54,905 workflow INFO:
	 Finished submitting: Run_bayes.a0002 ID: 4
170725-

170725-13:49:59,464 workflow INFO:
	 Finished submitting: Run_bayes.a0308 ID: 52
170725-13:49:59,466 workflow INFO:
	 Submitting: Run_bayes.a1673 ID: 53
170725-13:49:59,551 workflow INFO:
	 Finished submitting: Run_bayes.a1673 ID: 53
170725-13:49:59,553 workflow INFO:
	 Submitting: Run_bayes.a0685 ID: 54
170725-13:49:59,641 workflow INFO:
	 Finished submitting: Run_bayes.a0685 ID: 54
170725-13:49:59,643 workflow INFO:
	 Submitting: Run_bayes.a0509 ID: 55
170725-13:49:59,730 workflow INFO:
	 Finished submitting: Run_bayes.a0509 ID: 55
170725-13:49:59,732 workflow INFO:
	 Submitting: Run_bayes.a0723 ID: 56
170725-13:49:59,819 workflow INFO:
	 Finished submitting: Run_bayes.a0723 ID: 56
170725-13:49:59,820 workflow INFO:
	 Submitting: Run_bayes.a0307 ID: 57
170725-13:49:59,910 workflow INFO:
	 Finished submitting: Run_bayes.a0307 ID: 57
170725-13:49:59,912 workflow INFO:
	 Submitting: Run_bayes.a1363 ID: 58
170725-13:50:00,0 workflow INFO:
	 Finished submitting: Run_bayes.a1363 ID: 58
170

170725-13:50:04,341 workflow INFO:
	 Finished submitting: Run_bayes.a0079 ID: 106
170725-13:50:04,343 workflow INFO:
	 Submitting: Run_bayes.a1663 ID: 107
170725-13:50:04,432 workflow INFO:
	 Finished submitting: Run_bayes.a1663 ID: 107
170725-13:50:04,434 workflow INFO:
	 Submitting: Run_bayes.a1438 ID: 108
170725-13:50:04,520 workflow INFO:
	 Finished submitting: Run_bayes.a1438 ID: 108
170725-13:50:04,521 workflow INFO:
	 Submitting: Run_bayes.a0078 ID: 109
170725-13:50:04,609 workflow INFO:
	 Finished submitting: Run_bayes.a0078 ID: 109
170725-13:50:04,610 workflow INFO:
	 Submitting: Run_bayes.a1178 ID: 110
170725-13:50:04,773 workflow INFO:
	 Finished submitting: Run_bayes.a1178 ID: 110
170725-13:50:04,775 workflow INFO:
	 Submitting: Run_bayes.a0380 ID: 111
170725-13:50:04,860 workflow INFO:
	 Finished submitting: Run_bayes.a0380 ID: 111
170725-13:50:04,861 workflow INFO:
	 Submitting: Run_bayes.a1348 ID: 112
170725-13:50:04,947 workflow INFO:
	 Finished submitting: Run_bayes.a1

170725-13:51:01,281 workflow ERROR:
	 [u'Node Run_bayes.a1540 failed to run on host node002.cm.cluster.']
170725-13:51:01,283 workflow INFO:
	 Saving crash info to /storage/gablab001/data/genus/GIT/genus/bayes/notebooks/crash-20170725-135101-ysa-Run_bayes.a1540-39fa562e-5521-4644-ba75-f9620ca8ea79.pklz
170725-13:51:01,286 workflow INFO:
	 Traceback (most recent call last):
  File "/om/scratch/Tue/ysa/brain_bcv/batch/pyscript_20170725_134958_brain_bcv_Run_bayes.a1540.py", line 32, in <module>
    result = info['node'].run(updatehash=info['updatehash'])
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 482, in _run_interface
    self._result = self._run_command(execute)
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 613, in 

170725-13:51:08,371 workflow INFO:
	 Pending[148] Submitting[2] jobs Slots[2]
170725-13:51:08,372 workflow INFO:
	 Submitting: Run_bayes.a0057 ID: 150
170725-13:51:08,546 workflow INFO:
	 Finished submitting: Run_bayes.a0057 ID: 150
170725-13:51:08,548 workflow INFO:
	 Submitting: Run_bayes.a0284 ID: 151
170725-13:51:08,636 workflow INFO:
	 Finished submitting: Run_bayes.a0284 ID: 151
170725-13:52:48,256 workflow ERROR:
	 [u'Node Run_bayes.a1150 failed to run on host node002.cm.cluster.']
170725-13:52:48,258 workflow INFO:
	 Saving crash info to /storage/gablab001/data/genus/GIT/genus/bayes/notebooks/crash-20170725-135248-ysa-Run_bayes.a1150-daaa1435-f9db-4227-a4a8-59a9704912a5.pklz
170725-13:52:48,262 workflow INFO:
	 Traceback (most recent call last):
  File "/om/scratch/Tue/ysa/brain_bcv/batch/pyscript_20170725_135002_brain_bcv_Run_bayes.a1150.py", line 32, in <module>
    result = info['node'].run(updatehash=info['updatehash'])
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.

170725-13:52:52,737 workflow INFO:
	 Pending[149] Submitting[1] jobs Slots[1]
170725-13:52:52,739 workflow INFO:
	 Submitting: Run_bayes.a1482 ID: 152
170725-13:52:52,825 workflow INFO:
	 Finished submitting: Run_bayes.a1482 ID: 152
170725-13:52:58,117 workflow ERROR:
	 [u'Node Run_bayes.a0510 failed to run on host node002.cm.cluster.']
170725-13:52:58,119 workflow INFO:
	 Saving crash info to /storage/gablab001/data/genus/GIT/genus/bayes/notebooks/crash-20170725-135258-ysa-Run_bayes.a0510-9cbc18b3-a3e3-4a3f-908b-22913d3f5efe.pklz
170725-13:52:58,121 workflow INFO:
	 Traceback (most recent call last):
  File "/om/scratch/Tue/ysa/brain_bcv/batch/pyscript_20170725_134959_brain_bcv_Run_bayes.a0510.py", line 32, in <module>
    result = info['node'].run(updatehash=info['updatehash'])
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-p

170725-13:52:58,318 workflow ERROR:
	 [u'Node Run_bayes.a0639 failed to run on host node002.cm.cluster.']
170725-13:52:58,320 workflow INFO:
	 Saving crash info to /storage/gablab001/data/genus/GIT/genus/bayes/notebooks/crash-20170725-135258-ysa-Run_bayes.a0639-941403bb-202c-4f0a-9c8e-26b3de6a4c19.pklz
170725-13:52:58,323 workflow INFO:
	 Traceback (most recent call last):
  File "/om/scratch/Tue/ysa/brain_bcv/batch/pyscript_20170725_134959_brain_bcv_Run_bayes.a0639.py", line 32, in <module>
    result = info['node'].run(updatehash=info['updatehash'])
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 482, in _run_interface
    self._result = self._run_command(execute)
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 613, in 

170725-13:53:05,74 workflow INFO:
	 Pending[148] Submitting[2] jobs Slots[2]
170725-13:53:05,76 workflow INFO:
	 Submitting: Run_bayes.a1562 ID: 153
170725-13:53:05,163 workflow INFO:
	 Finished submitting: Run_bayes.a1562 ID: 153
170725-13:53:05,164 workflow INFO:
	 Submitting: Run_bayes.a0513 ID: 154
170725-13:53:05,247 workflow INFO:
	 Finished submitting: Run_bayes.a0513 ID: 154


KeyboardInterrupt: 